In [ ]:
import plotly.graph_objects as go
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import plotly.express as px
from EC_data_processing_lib import get_header_length, get_decimal_separator
from Data_setup_CHEM import data_set  # choose the correct python file
import os
from galvani import BioLogic
from datetime import datetime

# Automatically fix file paths in the data_set dictionary
for key, entry in data_set.items():
    for field in ['data_PEIS', 'data_CV', 'data_CA', 'data_CP', 'data_Tafel']:
        if entry[field]:
            entry[field] = entry[field].replace('\\', '/')

# Base title for file names (do not display on plot)
fig_title = 'Title'
today_str = datetime.today().strftime('%Y-%m-%d')

# Create two figures:
# - Potential Step: current (absolute, normalized) vs time
# - Cottrell: current (absolute, normalized) vs t^(-1/2) with linear regression annotation
fig_step = go.Figure()
fig_cottrell = go.Figure()

colors = px.colors.qualitative.Dark24

# --- Determine Valid Datasets ---
# Build a list of keys for which the file exists and contains a valid current column.
valid_keys = []
for key in data_set:
    file = data_set[key]['data_CA']
    if not os.path.exists(file):
        continue
    # Read the file once (depending on its extension)
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        try:
            data_temp = pd.read_csv(
                file,
                encoding='ansi',
                sep="\t",
                decimal=get_decimal_separator(file),
                skiprows=range(headerlength)
            )
        except Exception:
            continue
    else:
        try:
            mpr_file = BioLogic.MPRfile(file)
            data_temp = pd.DataFrame(mpr_file.data)
        except Exception:
            continue
    # Check if a valid current column exists
    if '<I>/mA' in data_temp.columns or 'I/mA' in data_temp.columns:
        valid_keys.append(key)

total_valid = len(valid_keys)
valid_index = 0  # to keep track of annotation position

# --- Process Each Valid Dataset ---
for key in valid_keys:
    file = data_set[key]['data_CA']
    
    # Read data from file
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(
            file,
            encoding='ansi',
            sep="\t",
            decimal=get_decimal_separator(file),
            skiprows=range(headerlength)
        )
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
        
    # Automatically select the correct current column
    if '<I>/mA' in data.columns:
        current = data['<I>/mA']
    elif 'I/mA' in data.columns:
        current = data['I/mA']
    else:
        continue  # Skip if neither column is present

    # Use the absolute value of the current and normalize by the sample mass
    current = np.abs(current)
    current_mass = current / data_set[key]['mass']
    
    # Get time data and compute t^(-1/2)
    time = data['time/s']
    time_root = time ** (-0.5)
    
    # --- Potential Step Figure: Current vs Time ---
    fig_step.add_trace(go.Scatter(
        x=time,
        y=current_mass,
        name=data_set[key]['label'],
        mode='markers',
        marker=dict(color=colors[data_set[key]['color_index']])
    ))
    
    # --- Cottrell Figure: Current vs t^(-1/2) ---
    fig_cottrell.add_trace(go.Scatter(
        x=time_root,
        y=current_mass,
        name=data_set[key]['label'],
        mode='markers',
        marker=dict(color=colors[data_set[key]['color_index']])
    ))
    
    # --- Linear Fit for the Cottrell Plot ---
    x_vals = time_root.to_numpy()
    y_vals = current_mass.to_numpy()
    
    # Linear regression: y = slope*x + intercept
    coeffs = np.polyfit(x_vals, y_vals, 1)
    slope, intercept = coeffs
    y_fit = slope * x_vals + intercept
    
    # Sort the data for a smooth fit line
    sort_idx = np.argsort(x_vals)
    x_fit = x_vals[sort_idx]
    y_fit = y_fit[sort_idx]
    
    # Calculate R² (coefficient of determination)
    ss_res = np.sum((y_vals - (slope * x_vals + intercept)) ** 2)
    ss_tot = np.sum((y_vals - np.mean(y_vals)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    
    # Add the linear fit line to the Cottrell figure
    fig_cottrell.add_trace(go.Scatter(
        x=x_fit,
        y=y_fit,
        mode='lines',
        line=dict(color=colors[data_set[key]['color_index']], dash='dash'),
        name=data_set[key]['label'] + ' fit'
    ))
    
    # --- Add Annotation (with non-overlapping y-position) ---
    # Compute y position: evenly spaced between 0 and 1 in paper coordinates.
    annotation_y = 1 - (valid_index + 1) / (total_valid + 1)
    valid_index += 1
    
    fig_cottrell.add_annotation(
        x=0.05,
        y=annotation_y,
        xref='paper',
        yref='paper',
        text=(f"{data_set[key]['label']} fit:<br>"
              f"y = {slope:.3f}x + {intercept:.3f}<br>"
              f"R² = {r2:.3f}"),
        showarrow=False,
        font=dict(color=colors[data_set[key]['color_index']], size=14),
        align='left',
        bordercolor='black',
        borderwidth=1,
        bgcolor='white',
        opacity=0.8
    )

# --- Update Layout for the Potential Step Figure using autoscaling ---
fig_step.update_xaxes(
    autorange=True,
    nticks=10,
    ticks='outside',
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    zeroline=False,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)
fig_step.update_yaxes(
    autorange=True,
    nticks=10,
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    ticks='outside',
    zeroline=True,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)
fig_step.update_layout(
    xaxis_title="Time (s)",
    yaxis_title="Current (mA/cm<sup>2</sup>)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
    ),
    title_text="",
    plot_bgcolor="rgba(0, 0, 0, 0)",
    paper_bgcolor="rgba(255, 255, 255, 255)",
    width=750,
    height=500,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1
    ),
    margin={'t': 0, 'l': 0, 'b': 0, 'r': 0}
)

# --- Update Layout for the Cottrell Figure using autoscaling ---
fig_cottrell.update_xaxes(
    autorange=True,
    nticks=10,
    ticks='outside',
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    zeroline=False,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)
fig_cottrell.update_yaxes(
    autorange=True,
    nticks=10,
    showgrid=False,
    gridwidth=1,
    gridcolor='grey',
    ticks='outside',
    zeroline=True,
    zerolinecolor='grey',
    zerolinewidth=1,
    showline=True,
    linewidth=2,
    linecolor='black'
)
fig_cottrell.update_layout(
    xaxis_title="t<sup>-1/2</sup>",
    yaxis_title="Current (mA/cm<sup>2</sup>)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
    ),
    title_text="",
    plot_bgcolor="rgba(0, 0, 0, 0)",
    paper_bgcolor="rgba(255, 255, 255, 255)",
    width=750,
    height=500,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1
    ),
    margin={'t': 40, 'l': 0, 'b': 0, 'r': 0}
)

# --- Display the Figures (optional) ---
fig_step.show()
fig_cottrell.show()

# --- Save the Figures ---
# Save the SVG files first (transparent background)
fig_step.write_image(f"{today_str} {fig_title} Step.svg")
fig_cottrell.write_image(f"{today_str} {fig_title} Cottrell.svg")

# Update layout for a white background (for JPEG saving and display)
fig_step.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 255)"
})
fig_cottrell.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 255)"
})

fig_step.write_image(f"{today_str} {fig_title} Step.jpeg")
fig_cottrell.write_image(f"{today_str} {fig_title} Cottrell.jpeg")
